In [ ]:
import os
import io
import datetime
import logging
import sys

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 

import civis
import civis.io
from civis.futures import CivisFuture

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, RandomForestRegressor
from civis.ml import ModelPipeline

import uuid
import json
from pprint import pprint
import tempfile
import concurrent.futures
from concurrent.futures import wait
from collections import namedtuple

In [ ]:
# Default feature lists for Rainbow Modeling Frame (each number corresponds to number of features)
feature_table = civis.io.read_civis_sql(sql='select * from bernie_nmarchio2.feature_list order by sort_order asc', use_pandas = True, database='Bernie 2020')
feature_list_large = list(feature_table[(feature_table['frame_large'] == 1)]['feature_name']) + ['state_code']
feature_list_medium = list(feature_table[(feature_table['frame_medium'] == 1)]['feature_name']) + ['state_code']
feature_list_small = list(feature_table[(feature_table['frame_small'] == 1)]['feature_name']) + ['state_code']

table_columns = civis.io.read_civis_sql(
    sql=f'''select ordinal_position as position, column_name, data_type 
    from information_schema.columns 
    where table_name = 'rainbow_modeling_frame' and table_schema = 'bernie_data_commons' and column_name != 'person_id'
    order by ordinal_position;''', use_pandas = True, database='Bernie 2020')

#exclusion_list_466 = [e for e in list(table_columns['column_name']) if e not in feature_list_466] 

In [ ]:
# Set Parameters
DATABASE = 'Bernie 2020'

# Primary key in both the DV table and the Modeling Frame
PRIMARY_KEY = 'person_id'

# Table containing recoded Dependent Variables keyed to the PRIMARY_KEY
DV_TABLE = 'bernie_nmarchio2.spoke_dvs'
# List of binarized dependent variables (accepts 1, 0, and null values) in DV_TABLE
DV_LIST = ['spoke_support_1box', 'spoke_persuasion_1plus', 'spoke_persuasion_1minus']


# Table containing covariates and keyed to PRIMARY_KEY
MODELING_FRAME = 'bernie_data_commons.rainbow_modeling_frame'
# Columns in the Modeling Frame to exclude from feature list (i.e., strings or incomplete coverage)
EXCLUSION_COLUMNS = ['state_code']

# Schema to contain prediction tables
SCHEMA = 'bernie_nmarchio2'
# String that will be concatenated in front of the output table's name
PREFIX = 'scored'


In [ ]:
datestamp = '{:%Y%m%d}'.format(datetime.date.today())

In [ ]:
# Counts of positive and negative classes
dv_sql_targets = "\n,".join(["sum({dv}) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_targets = f"""select {dv_sql_targets} from {DV_TABLE};"""
sql_count_targets = civis.io.read_civis_sql(sql_collapse_targets, DATABASE)

dv_sql_zeroes = "\n,".join(["sum(case when {dv} = 0 then 1 end) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_zeroes = f"""select {dv_sql_zeroes} from {DV_TABLE};"""
sql_count_zeroes = civis.io.read_civis_sql(sql_collapse_zeroes, DATABASE)


In [ ]:
# Determing training table proportion of positives to negatives (to avoid class imbalance problems)
sample_share = []
for i in range(len(DV_LIST)):
    u = round((int(sql_count_targets[1][i])*2)/(int(sql_count_zeroes[1][i])),5)
    sample_share.append(u)

In [ ]:
# Create training views
for i in range(len(DV_LIST)):
    if (int(sql_count_targets[1][i])*3) <= 1000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
    if (int(sql_count_targets[1][i])*3) > 1000 & (int(sql_count_targets[1][i])*3) <= 2000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_medium])
    if (int(sql_count_targets[1][i])*3) > 2000:
        feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_large])
    dv_item = DV_LIST[i]
    random_sample = sample_share[i]
    training_sql = f"""DROP VIEW IF EXISTS {SCHEMA}.{PREFIX}_training_{i} CASCADE;
    CREATE VIEW {SCHEMA}.{PREFIX}_training_{i} AS 
    (select * from (
    (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 1) 
    union all 
    (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 and random() < {random_sample}))
    left join
    (select {PRIMARY_KEY}, {feature_select} from {MODELING_FRAME}) using({PRIMARY_KEY}));"""
    create_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
    create_training_sql.result().state
 

In [ ]:
# Train models
train_list = []
model_list = []

for index, dv in enumerate(DV_LIST):
    print('TRAINING >>> {}'.format(dv))
    
    exc_list = DV_LIST.copy()
    exc_list.remove(dv)
    
    assert dv not in exc_list 
    
    name = f"""{dv}_{datestamp}"""
    model = ModelPipeline(model='sparse_logistic',
                          dependent_variable=dv,
                          primary_key=PRIMARY_KEY,
                          excluded_columns=EXCLUSION_COLUMNS,
                          calibration='sigmoid',
                          model_name=name,
                          memory_requested=12000)
    
    where_string = '{} is not null'.format(dv)

    train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training_{index}""", 
                        database_name=DATABASE,
                        sql_where=where_string#,
                        #fit_params={'sample_weight': WEIGHT_VAR}
                       )
    
    model_list.append(model)
    train_list.append(train)    


In [ ]:
# Extract successful models
model_output = model_list
train_output = train_list

jobs_list = []
for t in train_output: 
    try:
        if len(t.metadata['output']) > 0:  
            jobs_list.append(t)
            print('Job success')
    except:
        print('Job failure')
        pass

model_output, train_ouput = zip(*((model, train) for model, train in zip(model_output, train_output) if train in jobs_list))
model_output = list(model_output)
train_output = list(train_output)

In [ ]:
# Generate validation metrics
metrics_list = []

for a, b in enumerate(train_output):
    metric = {'job_id':b.job_id,
              'run_id':b.run_id,
              'dv': ''.join(b.metadata['run']['configuration']['data']['y']),
              'model': b.metadata['model']['model'],
              'time_of_train_run': b.metadata['run']['time_of_run'],
              'n_rows': b.metadata['data']['n_rows'],
              'n_features': b.metadata['data']['n_cols'],
              'auc': b.metadata['metrics']['roc_auc'],
              'deciles': b.metadata['metrics']['deciles'],
              'confusion_matrix': b.metadata['metrics']['confusion_matrix'],
              'accuracy': b.metadata['metrics']['accuracy'],
              'p_correct': b.metadata['metrics']['p_correct'],
              'pop_incidence_true': b.metadata['metrics']['pop_incidence_true'],
              'feature_list':b.metadata['model']['parameters']['relvars']
             }
    metrics_list.append(metric)
    
metric_order = (['job_id', 'run_id', 'dv', 'model', 'time_of_train_run', 'n_rows', 'n_features',
                 'auc', 'deciles', 'confusion_matrix', 'accuracy', 'p_correct','pop_incidence_true','feature_list'])

validation_df = pd.DataFrame.from_records(metrics_list, columns=metric_order, index='run_id')
validation_df

In [ ]:
# Write validation metrics to Redshift
create_validation_table = civis.io.dataframe_to_civis(df=validation_df,
                                                 database=DATABASE, 
                                                 table= f'{SCHEMA}.model_validation_{datestamp}', 
                                                 existing_table_rows='drop')


In [ ]:
# Score the voterfile
scores_list = []
for m,t in zip(model_output, train_output):
    DV_NAME = ''.join(t.metadata['run']['configuration']['data']['y'])
    print(DV_NAME)
    SCORES_TABLE = f'{SCHEMA}.{PREFIX}_{DV_NAME}_{datestamp}'
    scores_list.append(SCORES_TABLE)
    scores = m.predict(primary_key=PRIMARY_KEY,
                       database_name=DATABASE, 
                       table_name=MODELING_FRAME,
                       if_exists='drop',
                       output_table=SCORES_TABLE,
                       disk_space=20)
scores.result()


In [ ]:
# Generate SQL for final output table and drop intermediary tables
view_list = []
table_list = []
for i in range(len(DV_LIST)):
    view = f"{SCHEMA}.{PREFIX}_training_{i}"
    view_list.append(view)
    table = f"{SCHEMA}.{PREFIX}_{DV_LIST[i]}_{datestamp}"
    table_list.append(table)

drop_view_sql = "\n".join(["drop view if exists {view} CASCADE;".format(view=v) for v in view_list])
drop_table_sql = "\n".join(["drop table if exists {tbl};".format(tbl=t) for t in table_list])  
dv_strings = "\n,".join(["{dv_score}_1 as {dv_score}".format(dv_score=dv) for dv in DV_LIST])
dv_tiles = "\n,".join(["NTILE(100) OVER (ORDER BY {dv_tile}_1) AS {dv_tile}_100".format(dv_tile=dv) for dv in DV_LIST])
join_table = []
if len(table_list) > 1:
    for i in table_list[1:]:
        j = str(' left join '+f'{i}'+f' using({PRIMARY_KEY}) ')
        join_table.append(j)
        #dv_strings = "\nleft join ".join(["{dv_score}".format(table=tbl) for tbl in table_list[i])


In [ ]:
output_table_sql = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_output_{datestamp};
CREATE TABLE {SCHEMA}.{PREFIX}_output_{datestamp}
  DISTSTYLE KEY
  DISTKEY ({PRIMARY_KEY})
  SORTKEY ({PRIMARY_KEY})
  AS ("""+'select '+ f"{PRIMARY_KEY} \n," + dv_strings + "\n," + dv_tiles + ' from '+ ''.join(table_list[0]) + ''.join(join_table) +');'  


In [ ]:
print(output_table_sql)

In [ ]:
# Create final output table
create_output_table = civis.io.query_civis(sql=output_table_sql, database=DATABASE)
create_output_table.result().state


In [ ]:
# Drop intermediary tables
drop_views_query = civis.io.query_civis(sql=drop_view_sql, database=DATABASE)
drop_views_query.result().state

drop_tables_query = civis.io.query_civis(sql=drop_table_sql, database=DATABASE)
drop_tables_query.result().state

In [ ]:
print(drop_view_sql)
print(drop_table_sql)

In [ ]:
# Grant team on tables
grant_statement = f"""
GRANT ALL ON SCHEMA {SCHEMA} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_output_{datestamp} TO GROUP bernie_data;
"""
grant_team = civis.io.query_civis(sql=grant_statement, database=DATABASE)
grant_team.result().state

In [ ]:
print(grant_statement)